In [12]:
import pandas as pd

In [ ]:
def read_tbl():
    df = pd.read_excel("../data/input/Additional INC0090307.xlsx")

    df = df.rename(columns={"employeeid": "person_nbr", 
                            "BirthDate": "birth_date", 
                            "AgencyName": "agency_name",
                            "StartDate":"start_date", 
                            "EndDate":"end_date",
                            "separationReason":"separation_reason",
                            "Status": "employment_status",
                            "fullname": "full_name",
                            })

    return df 


def split_name(fullname):
    parts = fullname.split(',')
    last_name = parts[0].strip()
    suffix = parts[1].strip() if len(parts) > 2 else ''
    first_middle = parts[-1].strip().split(' ')
    first_name = first_middle[0].strip()
    middle_name = ' '.join(first_middle[1:]).strip() if len(first_middle) > 1 else ''
    return first_name, middle_name, last_name, suffix


def clean_name(df):
     df.loc[:, "suffix"] = df.suffix.str.replace(r"\.", "", regex=True)
     df.loc[:, "middle_name"] = df.middle_name.str.replace(r"\.", "", regex=True)
     return df 


def strip_time_from_dates(df):
        df['start_date'] = df['start_date'].dt.date
        df['end_date'] = df['end_date'].dt.date
        return df


def extract_birth_year(df):
     df.loc[:, "year_of_birth"] = df.birth_date.astype(str).str.replace(r"^(\w+)-(.+)", r"\1", regex=True)
     return df 


def clean_misc(df):
    df = df.drop(columns=["birth_date"])
    df = df.fillna("")
    return df 

def clean_sep_reason(df):
     df.loc[:, "separation_reason"] = df.separation_reason.str.lower().str.strip()
     return df 


df = read_tbl()
df[['first_name', 'middle_name', 'last_name', 'suffix']] = df['full_name'].apply(lambda x: pd.Series(split_name(x)))
df = df.pipe(strip_time_from_dates).pipe(extract_birth_year).pipe(clean_name).pipe(clean_misc).pipe(clean_sep_reason)



In [14]:
df.agency_name.unique().tolist()

['Reynoldsburg Police Department',
 'Goshen Twp. Police Department - Mahoning',
 'Mill Creek MetroParks Police Department',
 "Mahoning County Sheriff's Office",
 'Poland Twp. Police Department',
 'Indian Hill Police Department',
 'Sandusky Police Department',
 'Sandusky Police/Cedar Point Division',
 'Dalton Police Department',
 'Summa Health Protective Services/Police',
 "Stark County Sheriff's Office",
 'Canton Police Department',
 'Akron Police Department',
 'Warrensville Heights Police Department',
 'Hinckley Police Department',
 'Newburgh Heights Police Department',
 'Columbus Police Department',
 'Clinton County Sheriffs Office',
 "Clark County Sheriff's Office",
 "Hancock County Sheriff's Office",
 'Geneva-on-the-Lake Police Department',
 "Ashtabula County Sheriff's Office",
 'Geneva Police Department',
 'Orwell Police Department',
 'Millersport Police Department',
 'Ohio Dept. of Natural Resources - Division of Parks and Recreation ',
 "Hocking County Sheriff's Office",
 'Ohio 

In [15]:
## done

df.to_csv("../data/output/ohio-processed.csv", index=False)